In [1]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
import mlflow

In [2]:
train = pd.read_csv('../data/train.csv')

In [3]:
X = train.drop(columns=[
    'ID', 'target1', 'target2', 'target3', 'target4', 'target5',
    'target6', 'target7', 'target8', 'target9', 'target10', 'target11'
])

y = train.drop(columns=[
    'ID', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6',
    'feature7', 'feature8', 'feature9', 'feature10', 'feature11',
    'feature12', 'feature13', 'feature14', 'feature15'
])


In [4]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Hyper-param optmization

In [5]:
import optuna
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
import mlflow
import numpy as np
import warnings
from sklearn.metrics import root_mean_squared_error

warnings.filterwarnings('ignore')

def objective(trial):
    with mlflow.start_run(run_name=f'Trial-{trial.number}', nested=True):
        params = {
            'alpha': trial.suggest_float('alpha', 1e-4, 10.0, log=True),
            'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
            'solver': trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']),
            'max_iter': trial.suggest_int('max_iter', 100, 5000),
            'tol': trial.suggest_float('tol', 1e-6, 1e-2, log=True),
            'random_state': 42
        }

        # Remove max_iter para solvers que não precisam
        if params['solver'] in ['svd', 'cholesky']:
            params.pop('max_iter')
            params.pop('tol')

        mlflow.log_params(params)

        base = Ridge(**params)
        model = MultiOutputRegressor(base, n_jobs=-1)

        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        rmses = []

        for train_idx, val_idx in kf.split(X):
            X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
            y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

            model.fit(X_tr, y_tr)

            y_pred = model.predict(X_va)
            rmses.append(root_mean_squared_error(y_va, y_pred))

        rmse_mean = np.mean(rmses)
        mlflow.log_metric('rmse', rmse_mean)
        return rmse_mean

c:\Users\Guilherme\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=20,
        n_warmup_steps=10,
        interval_steps=5
    )
)

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('DSWA - Competicao3')
with mlflow.start_run(run_name='Ridge optimization'):
    study.optimize(objective, n_trials=50)
    
    print("Melhores hiperparâmetros:")
    print(study.best_params)
    print(f"Melhor RMSE: {study.best_value:.4f}")

    # Loga os melhores parâmetros e métrica no MLflow
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_rmse", study.best_value)


[I 2025-07-29 15:20:02,448] A new study created in memory with name: no-name-377e62a5-8655-4a85-9a57-0e277927c4f9
[I 2025-07-29 15:20:08,596] Trial 0 finished with value: 0.5953885916218324 and parameters: {'alpha': 0.0074593432857265485, 'fit_intercept': True, 'solver': 'sparse_cg', 'max_iter': 200, 'tol': 0.00757947995334801}. Best is trial 0 with value: 0.5953885916218324.


🏃 View run Trial-0 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/b32e41aa0d0545829e1b555b36a0de38
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-29 15:20:09,726] Trial 1 finished with value: 0.5951115651259281 and parameters: {'alpha': 1.452824663751602, 'fit_intercept': True, 'solver': 'sag', 'max_iter': 1531, 'tol': 2.9204338471814107e-05}. Best is trial 1 with value: 0.5951115651259281.


🏃 View run Trial-1 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/07caa79c60cf40ffa675a31b641799f2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-29 15:20:10,151] Trial 2 finished with value: 0.5948233580073409 and parameters: {'alpha': 0.019069966103000432, 'fit_intercept': True, 'solver': 'saga', 'max_iter': 4832, 'tol': 0.0017123375973163992}. Best is trial 2 with value: 0.5948233580073409.


...

🏃 View run Trial-47 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/5b681f9ec3ef4f12b8f61f741f1e9f19
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-29 15:20:34,284] Trial 48 finished with value: 0.5911322563177795 and parameters: {'alpha': 0.001724192701153952, 'fit_intercept': True, 'solver': 'sparse_cg', 'max_iter': 788, 'tol': 1.1416388575443275e-06}. Best is trial 48 with value: 0.5911322563177795.


🏃 View run Trial-48 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/3db4a7670022463fab0a2a37af856e39
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-29 15:20:34,728] Trial 49 finished with value: 0.5959213461959253 and parameters: {'alpha': 0.020450367312082066, 'fit_intercept': False, 'solver': 'auto', 'max_iter': 1547, 'tol': 5.037162622848114e-06}. Best is trial 48 with value: 0.5911322563177795.


🏃 View run Trial-49 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/33259b9bbbae40aa82742a7f4fa36ca1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
Melhores hiperparâmetros:
{'alpha': 0.001724192701153952, 'fit_intercept': True, 'solver': 'sparse_cg', 'max_iter': 788, 'tol': 1.1416388575443275e-06}
Melhor RMSE: 0.5911
🏃 View run Ridge optimization at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/73d01853224940b99de1d066ce9fc586
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
